<a href="https://colab.research.google.com/github/aaaksenova/NLP-Homework/blob/main/Aksenova_NLP_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Key Words Homework

To-do:

(1 балл) Подготовить мини-корпус (не меньше 4 текстов, 
примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. Предполагается, что вы найдете источник текстов, в котором уже выделены ключевые слова. Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова. **(DONE)**

(2 балла) Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой. Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной). **(DONE)**

(2 балла) Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf\*idf, OKAPI BM25, ...) **(DONE)**

(2 балла) Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun). Применить эти фильтры к спискам ключевых слов.

(2 балла) Оценить точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них. **(DONE)**

(1 балл) Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем. **(DONE)**

In [ ]:
! pip install spacy --upgrade
! python -m spacy download ru_core_news_md
! pip install python-rake
! pip install summa
! pip install sentence-transformers

     |████████████████████████████████| 5.9 MB 8.3 MB/s 
     |████████████████████████████████| 451 kB 38.6 MB/s 
     |████████████████████████████████| 10.1 MB 38.3 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 628 kB 28.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 42.7 MB 2.3 kB/s 
     |████████████████████████████████| 55 kB 

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import re
import pandas as pd
import seaborn as sns
import RAKE
from summa import keywords
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Corpus Creation

In [ ]:
with open('key_papers.txt') as fh:
    fil = fh.read()

In [ ]:
papers = fil.split('Литература\n')[2:]

In [ ]:
len(papers)

26

In [ ]:
key_to_paper = [(i.split('Ключевые слова: ')[1].split('Key words:')[0], 
                 i.split('Ключевые слова: ')[1].split('Key words:')[1]) for i in papers[:-1]]

In [ ]:
key_words = []
cleaned_papers = []
for key, paper in key_to_paper:
    key = key.strip().strip('.').replace('-\n', '').replace('\n', ' ')
    key_words.append([k for k in key.split(', ')])
    paper = re.sub(r'\[.+?\]', '', paper, flags=re.DOTALL)
    paper = re.sub(r'\(.+?\)', '', paper, flags=re.DOTALL)
    paper = re.sub(r'[A-Za-z,;1234567890]', '', paper)
    paper = paper.replace('\n', ' ')
    cleaned_papers.append(paper)

In [ ]:
df = pd.DataFrame({'key_words' : [', '.join(key) for key in key_words], 
                   'papers' : cleaned_papers})

In [ ]:
df = df.iloc[16:20]

In [ ]:
df.to_csv('keyword_data.csv', sep='\t')

## Keywords Manual Annotation

Я составляла свою разметку на основе ключевых слов, которые были предложены авторами. Из основных систематических изменений были следующие:

1. Добавлены аббревиатуры. Часто авторы вводят аббревиатуру в начале статьи и дальше используют только ее, а в ключевых словах указана полная форма (совершенный вид - СВ)

2. Все ключевые слова приведены к начальной форме: глаголы –> глагол

3. Некоторые ключевые слова объединяли два концепта, я их разделила (сложное и простое предложение –> сложное предложение, простое предложение)

Также добавила или убрала несколько, но большую часть слов оставила без изменений.

## Automated keyword extraction

In [ ]:
df = pd.read_csv('keyword_data.csv', sep='\t') # В этом файле уже исправленные мною ключевые слова

In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
model = spacy.load('ru_core_news_md')


def preprocess(text, model):

    docum = model(text)
    words = ' '.join([w.lemma_ for w in docum if w.lemma_.isalpha()])
    return words

In [ ]:
df['prep_texts'] = df.papers.apply(lambda x: preprocess(x, model))
df['prep_kw'] = df.papers.apply(lambda x: preprocess(x, model))

In [ ]:
df['key_count'] = df.key_words.apply(lambda x: len(x.split(', ')))

In [ ]:
df

### Rake

In [ ]:
stop = stopwords.words('russian') + ['который']

In [ ]:
rake = RAKE.Rake(stop)

In [ ]:
def kw_rake(text, count, rake):
    kw = rake.run(text, maxWords=3, minFrequency=3)
    kw = [k[0] for k in kw]
    return ', '.join(kw[:count])

In [ ]:
df['rake_raw'] = df.apply(lambda x: kw_rake(x['papers'], x['key_count'], rake), axis=1)
df['rake_prep'] = df.apply(lambda x: kw_rake(x['prep_texts'], x['key_count'], rake), axis=1)

### TextRank

In [ ]:
def kw_textrank(text, count):
    kw = keywords.keywords(text, language='russian', additional_stopwords=stop, scores=True)[:10]
    return ', '.join([k[0] for k in kw])

In [ ]:
df['textrank'] = df.apply(lambda x: kw_textrank(x['prep_texts'], x['key_count']), axis=1)

### Embedding-based

Based on: https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea 

In [ ]:
model = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
def kw_embeddings(text, model, top_n):
    n_gram_range = (1, 2)
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop).fit([text])
    candidates = count.get_feature_names()
    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    return ', '.join(keywords)

In [ ]:
df['emb_raw'] = df.apply(lambda x: kw_embeddings(x['prep_texts'], model, x['key_count']), axis=1)

In [ ]:
df['emb_prep'] = df.apply(lambda x: kw_embeddings(x['papers'], model, x['key_count']), axis=1)

## Syntactic patterns for keywords

## Evaluation metrics

Давайте представим тексты как набор токенов и их сочетаний. Тогда задачу извлечения ключевых слов можно свести к бинарной классификации: каждый элемент набора либо является, либо не является ключевым словом. Исходя из такой трансформации посчитаем метрики

In [ ]:
df_metrics = pd.DataFrame(columns = ['text_id' , 'extraction_algorithm', 'precision' , 'recall', 'f-score'])

for text_id in range(df.shape[0]):
    attributes = {column: df.iloc[text_id][column] for column in df if 'raw' not in column}
    count = CountVectorizer(ngram_range=(1, 2), stop_words=stop).fit([attributes['papers']])
    df_raw_tokens = pd.DataFrame({'tokens' : count.get_feature_names()})
    for i in ['prep_kw', 'rake_prep', 'textrank', 'emb_prep']:
        df_raw_tokens[i] = df_raw_tokens['tokens'].apply(lambda x: 1 if x in attributes[i] else 0)
    for i in ['rake_prep', 'textrank', 'emb_prep']:
        precision, recall, fscore, support = precision_recall_fscore_support(df_raw_tokens['prep_kw'], 
                                                                             df_raw_tokens[i], average='binary')
        df_metrics = df_metrics.append(pd.Series((text_id, i, precision, recall, fscore), 
                                                 index=df_metrics.columns), ignore_index=True)

In [ ]:
cm = sns.light_palette("purple", as_cmap=True)
pd.pivot_table(df_metrics, index=['text_id', 'extraction_algorithm']).style.background_gradient(cmap=cm)

## Final discussion

1. ОБЗОР МЕТРИК ПО ТАБЛИЧКЕ

2. Судя по всему, существуют более адекватные метрики для извлечения ключевых слов и в датасетах, которые сделаны для этой задачи помимо наличия слова в списке учитывается "уверенность" в этом слове, то есть его ранг. Про метрики я читала [тут](https://medium.com/gumgum-tech/exploring-different-keyword-extractors-evaluation-metrics-and-strategies-ef874d336773)

3. Надо как-то получше предобработать данные, возможно, попробовать стемминг, потому что в одной из статей писали об обособлении и смысл выражался разными частями речи (обособлять, обособление, обособленный)

4. Можно было бы сделать кластеризацию эмбеддингов всех кандидатов на ключевые слова и искать близость не отдельных токенов к тексту, а центроидов или просто средних векторов кластеров. А потом брать слово, которое ближе всего к вектору, репрезентующему кластер

5. Вообще эмбеддинги тут хороши, потому что часто авторы статьи выносят в ключевые слова какие-то обобщающие понятия, например, у статьи в журнале может быть тег NLP, но при этом она будет только о NER и сам токен NLP не встретится ни разу. Поэтому, возможно, для метрики качества выделения ключевых слов, возможо, тоже можно пробовать косинусное расстояние, если мы хотим понять, способен ли наш алгоритм выделять ключевые смыслы (которые по сути упрощаются до ключевых слов)